# A Simple, DeepBugs-like Deep Learning-Based Bug Detector

## Goal

This interactive tutorial shows how to learn a simple, [DeepBugs](https://github.com/michaelpradel/DeepBugs)-like bug detector that finds [argument swapping bugs](http://software-lab.org/publications/oopsla2017.pdf). This kind of bug occurs when accidentally swapped the arguments passed to a function, e.g., writing `setPoint(y, x)` instead of `setPoint(x, y)`.


## Loading the Data

Get a small training data set for finding swapped function arguments:

In [ ]:
!gdown https://drive.google.com/uc?id=1w7KZ5ugVfcz4RGsY2feSalH9xpa_JFLh


Downloading...
From: https://drive.google.com/uc?id=1w7KZ5ugVfcz4RGsY2feSalH9xpa_JFLh
To: /content/DeepBugs_data.tar.gz
20.1MB [00:00, 64.2MB/s]


In [ ]:
!tar -xzf DeepBugs_data.tar.gz

In [ ]:
!ls DeepBugs_data/calls

calls_1523347143670.json  calls_1523347146998.json  calls_1523347147613.json
calls_1523347145433.json  calls_1523347147049.json  calls_1523347147639.json
calls_1523347145539.json  calls_1523347147069.json
calls_1523347145593.json  calls_1523347147513.json


The data are function calls extracted from open-source JavaScript code. Let's read the JSON data into our Python-based learning code:

In [ ]:
import os
import json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout

In [ ]:
calls = []
for file in os.listdir("DeepBugs_data/calls"):
  with open(os.path.join("DeepBugs_data/calls", file)) as fp:
    calls.extend(json.load(fp))

print(f"Have read {len(calls)} function calls")

Have read 28005 function calls


We'll also use a pre-trained embedding of code tokens. It's a Word2Vec model trained on tokenized JavaScript code.

In [ ]:
with open("DeepBugs_data/token_to_vector.json") as fp:
  token_to_vector = json.load(fp)

print(f"Have loaded {len(token_to_vector)} token embeddings.")

Have loaded 9930 token embeddings.


## Preparing the Data: From Code Snippets to Vectors

We will train a model that distinguishes correct from buggy function calls. To this end, we prepare positive (i.e., correct) and negative (i.e., buggy) code examples. The positive examples are function calls as found in the open-source code (assuming that most code is correct). We create negative examples by swapping function arguments, i.e., artificially creating buggy code.

The following also represents each function call as a vector. The vector representation consists of the token embeddings of
 * the called function, i.e., the callee,
 * the first argument, and
 * the second argument.

In [ ]:
xs = []   # Inputs given to the model: Each element is
          #   the vector representation of a function call.
ys = []   # Outputs expected from the model: For each
          #   call, predict the probability that it's buggy.

for call in calls:
  if (call["callee"] in token_to_vector and
      call["arguments"][0] in token_to_vector and
      call["arguments"][1] in token_to_vector):
    callee_vec = token_to_vector[call["callee"]]
    arg1_vec = token_to_vector[call["arguments"][0]]
    arg2_vec = token_to_vector[call["arguments"][1]]

    # Positive, i.e., correct example
    x_correct = callee_vec + arg1_vec + arg2_vec
    # Negative, i.e., buggy example
    x_buggy = callee_vec + arg2_vec + arg1_vec

    xs.append(x_correct)
    ys.append(0)  # Probability that buggy is 0
    xs.append(x_buggy)
    ys.append(1)  # Probability that buggy is 1

# Split into training and validation data
nb_training = int(0.9*len(xs))
xs_training = np.array(xs[:nb_training])
ys_training = np.array(ys[:nb_training])
xs_validation = np.array(xs[nb_training:])
ys_validation = np.array(ys[nb_training:])

print(f"{len(xs_training)} training examples")
print(f"{len(xs_validation)} validation examples")

21592 training examples
2400 validation examples


## Training the Model

We train a simple feedforward model that takes the vector representation of a call and predicts the probability that the call is buggy.

In [ ]:
x_length = len(xs[0])
model = Sequential()
model.add(Dropout(0.2, input_shape=(x_length,)))
model.add(Dense(200, input_dim=x_length, activation="relu", kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid", kernel_initializer='normal'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(xs_training, ys_training, batch_size=100, epochs=5, verbose=1)        

Epoch 1/5
21592/21592 [==============================] - 1s 49us/step - loss: 0.5058 - accuracy: 0.7411
Epoch 2/5
21592/21592 [==============================] - 1s 45us/step - loss: 0.3610 - accuracy: 0.8311
Epoch 3/5
21592/21592 [==============================] - 1s 44us/step - loss: 0.2969 - accuracy: 0.8650
Epoch 4/5
21592/21592 [==============================] - 1s 44us/step - loss: 0.2533 - accuracy: 0.8883
Epoch 5/5
21592/21592 [==============================] - 1s 44us/step - loss: 0.2282 - accuracy: 0.8991


In [ ]:
validation_stats = model.evaluate(xs_validation, ys_validation)
print(f"Validation accuracy: {validation_stats[1]}")

2400/2400 [==============================] - 0s 36us/step
Validation accuracy: 0.8383333086967468


## Using the Learned Bug Detection Model

Once trained, we can query the model with a given function call. In a full implementation, the model would reason about calls extracted from JavaScript code. Here, we simply give the callee and arguments as a string:

In [ ]:
# Function call: setTimeout(delay, fn)
callee = "ID:setTimeout"  # Prefix "ID:" is to indicate that it's an identifier.
arg1 = "ID:delay"
arg2 = "ID:fn"

x = token_to_vector[callee] + token_to_vector[arg1] + token_to_vector[arg2]
xs = np.array([x])

buggy_probabilities = model.predict(xs)
print(f"Call is buggy with probability {str(round(buggy_probabilities[0][0], 4))}")

Call is buggy with probability 0.9904


The above call is indeed buggy, because `setTimeout`, one of the built-in functions in JavaScript, expects a function as the first argument and the delay after which to call the function as the second argument.

## Exercises

A few questions and tasks to check your understanding of this code. Modify the above code to answer them.
 * Inspect the provided JSON data. What other properties of function calls are there that may be useful for learning a bug detector?
 * How long is the vector that embeds a single token?
 * Does the model get any better when training for more epochs?
 * If you fix the call to `setTimeout`, i.e., pass the arguments in the correct order, what's the predicted probability that the code is buggy?